# Import modules
CDC data set description link [here](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2015)

In [5]:
import pdb
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, SelectKBest, mutual_info_classif, mutual_info_regression

import nhanes as nhanes

%matplotlib notebook

## Settings

In [6]:
DATA_PATH = 'C:/Users/allen/Documents/Git-Repos/Opportunistic/CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).

In [7]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_cancer()
n_fe = ds.features.shape[1]
n_classes = 2

Processing: Dietary\DR2TOT_H.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Processing: Laboratory\PAH_F.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Processing: Questionnaire\WHQ_H.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Extract the features and targets.

In [39]:
# indx = np.argwhere(ds.targets != 3)
dataset_features = ds.features # [ds.targets != 3]
dataset_targets = ds.targets # [ds.targets != 3]

# Pre-fix
# dataset_features = ds.features
# dataset_targets = ds.targets

## Preprocessing of Data
### Drop features with too-low variance

In [40]:
dataset_features_sel = dataset_features.copy()

# var_thresh = .001
# dropped_keys = []
# for key, value in dataset_features_sel.iteritems():
#     if(value.var() < var_thresh):
#         dataset_features_sel.drop(key, axis=1, inplace=True)
#         dropped_keys.append(key)

# print("Dropped %i keys, they were:\n %s" %(len(dropped_keys), dropped_keys))

### Seperate one-hot encoded, discrete, and continuous features

In [42]:
# onehot encoded
dataset_features_onehot = dataset_features_sel.filter(regex=(".*#.*"))
# Continuous 
dataset_features_cont = dataset_features_sel.drop(columns=dataset_features_sel.filter(regex=(".*#.*")))

dataset_features_disc = []
for key, value in dataset_features_cont.iteritems():
    # discrete were normalized so they should have a mean of .5, and we expect 3 of them
    if(value.mean() >= .5):
        dataset_features_disc.append(dataset_features_cont[key])
        dataset_features_cont.drop(key, axis=1, inplace=True)
# discrete
dataset_features_disc = pd.concat(dataset_features_disc, axis=1)
print(dataset_features_sel.shape)
print(dataset_features_cont.shape)
print(dataset_features_onehot.shape)
print(dataset_features_disc.shape)

(49509, 119)
(49509, 55)
(49509, 61)
(49509, 3)


### Calculate mutual information

In [ ]:
dataset_features_sel_vals = dataset_features_sel.values
dataset_targets_vals = np.ravel(dataset_targets.values) # unroll the column to the correct vector shape
mutual_info_cont = mutual_info_regression(dataset_features_cont.values, dataset_targets_vals)
mutual_info_disc = mutual_info_regression(dataset_features_disc.values, dataset_targets_vals)
mutual_info_onehot = mutual_info_classif(dataset_features_onehot.values, dataset_targets_vals)
# print(mutual_info)

### Convert mutual info into dataframes and inspect

In [ ]:
mutual_info_cont_df = pd.Dataframe(data=mutual_info_cont, columns=dataset_features_cont.columns())
mutual_info_disc_df = pd.Dataframe(data=mutual_info_disc, columns=dataset_features_disc.columns())
mutual_info_onehot_df = pd.Dataframe(data=mutual_info_onehot, columns=dataset_features_onehot.columns())

### Dropping features with low mutual information
    __Conditioned on target variable__

In [13]:
mi_df = pd.DataFrame(mutual_info)
mi_df = mi_df.transpose()
selector = mi_df > .01
print(mi_df)

to_drop = []
for row in selector.itertuples():
    for i in range(len(row)-1):
        if((row[i])):
            to_drop.append(i)

mu_features_sel = pd.DataFrame(dataset_features_sel.copy())
mu_features_sel = mu_features_sel.drop(columns=to_drop)
print(mu_features_sel.shape())

    0         1    2    3         4    5         6         7         8    9   \
0  0.0  0.042326  0.0  0.0  0.000296  0.0  0.010882  0.004473  0.010379  0.0   

   ...   83   84        85        86   87        88        89   90   91   92  
0  ...  0.0  0.0  0.005267  0.001397  0.0  0.003795  0.003083  0.0  0.0  0.0  

[1 rows x 93 columns]


KeyError: '[2 7 9 11] not found in axis'

## Train/Test Separation

In [ ]:
perm = np.random.permutation(dataset_targets.shape[0])
dataset_features = dataset_features_sel[perm]
dataset_targets = dataset_targets[perm]

print("dataset_features Shape: %s, dataset_targets Shape: %s" % (dataset_features.shape, dataset_targets.shape))

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]
    
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')

## Classification

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_RFC', accu)

clf = SVC(gamma='auto')
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_SVC', accu)

clf = LogisticRegression(solver='lbfgs', max_iter=200)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_LR', accu)


In [ ]:
print(classification_report(targets_tst, preds_tst))

### Accuracies from baseline: 
#### Cancer (ds.load_cancer()):
* accu_tst_RFC 0.758
* accu_tst_SVC 0.759
* accu_tst_LR 0.768

#### Arthiritis (ds.load_arthiritis()):
* accu_tst_RFC 0.753
* accu_tst_SVC 0.754
* accu_tst_LR 0.773